# Dokument-chattbot

In [1]:
import numpy as np
from google import genai
from pathlib import Path
import re

from google.genai import types

In [ ]:
import os
from dotenv import load_dotenv
load_dotenv() 
api_key = os.environ["GOOGLE_API_KEY"]
client = genai.Client(api_key=api_key)

response = client.models.generate_content(
    model="gemini-2.0-flash", 
    contents="Vad är planks konstant?"
)

print(response.text)

Planks konstant, betecknad med symbolen **h**, är en fundamental fysisk konstant som relaterar energin hos en foton (eller andra kvantiserade partiklar) till dess frekvens.

**Definition:**

*   **h = 6.62607015 × 10⁻³⁴ joule-sekunder (J⋅s)**
*   Ibland används även den reducerade Planck-konstanten (ℏ eller h-bar), som är lika med h/(2π):
    *   **ℏ = 1.054571817 × 10⁻³⁴ J⋅s**

**Betydelse:**

*   **Kvantisering:**  Planks konstant är central för kvantmekaniken. Den indikerar att energi, rörelsemängd och andra fysiska egenskaper ofta är "kvantiserade," vilket innebär att de bara kan anta diskreta, specifika värden.  Tänk på det som att energin inte kan öka eller minska kontinuerligt, utan bara i "hopp" eller "kvanta" vars storlek bestäms av Planks konstant.

*   **Fotonenergi:**  Energin (E) hos en foton är relaterad till dess frekvens (ν) genom formeln:
    *   **E = hν**
    *   Eller, med våglängd (λ) och ljusets hastighet (c):
    *   **E = hc/λ**

*   **Heisenbergs osäkerhetsprin

In [3]:
print("*** Gemini chat ***")
print("Type <q> to exit chat.")

while True:
    prompt = input("User: ")
    if prompt == "q":
        break
    else:
        response = client.models.generate_content(
            model="gemini-2.0-flash",
            contents=prompt
        )
        print("Gemini:", response.text)

*** Gemini chat ***
Type <q> to exit chat.
Gemini: Hello there! How can I help you today?

Gemini: Okay, exiting now. Goodbye!

Gemini: Please provide me with some context! I need to know what you'd like me to do with this blank prompt. For example, are you asking me to:

*   **Tell a story?** Give me a theme or a starting point.
*   **Write code?** Tell me what language and what you want the code to do.
*   **Answer a question?** Tell me what the question is.
*   **Suggest ideas?** Tell me what kind of ideas you're looking for.
*   **Provide a creative response?** Give me a general topic or feeling.

The more information you give me, the better I can respond to your request.



KeyboardInterrupt: Interrupted by user

In [ ]:
paths = sorted(Path("/Users/user/mai24ma/AI_theory_2/week_3/lesson_1").glob("*.txt")) 
text = "\n\n".join(p.read_text(encoding="utf-8", errors="ignore") for p in paths) # Read and concatenate text files

In [5]:
print(len(text))

24657


In [6]:
print(text[2000:7000])

m and Rosie's friends and relatives. Among them were Elanor the Fair, Frodo Gardner, and Goldilocks.

Fourth Age
In their later years, Sam and Rosie went to Gondor and lived there for over one year in the house of King Aragorn II. When they returned, they lived out their lives seeing Sam's mayoral duties passed to Tolman Cotton Jr. Rosie died in the year 61 on Mid-year's Day at the age of ninety-eight. Sam then left Middle-earth for the Undying Lands later that year after giving their daughter the Red Book of Westmarch.

Behind the scenes
"I think the simple ‘rustic’ love of Sam and his Rosie (nowhere elaborated) is absolutely essential to the study of his (the chief hero's) character, and to the theme of the relation of ordinary life (breathing, eating, working, begetting) and quests, sacrifice, causes, and the 'longing for Elves', and sheer beauty." - J.R.R. Tolkien letter dated 1951.

In adaptations
In the Rankin/Bass film
In The Return of the King (1980 film), Rosie was not voiced 

In [7]:
chunks = []
n = 500
overlap = 50
for i in range(0, len(text), n - overlap):
  chunks.append(text[i:i + n])

print(f"Number of chunks: {len(chunks)}")

Number of chunks: 55


In [8]:
print(chunks[0])

Title: Rose Cotton
Source: https://lotr.fandom.com/wiki/Rose_Cotton

Rose
Rose Cotton, nicknamed Rosie, was a hobbit of the Shire. She wedded Samwise Gamgee, renowned companion of Frodo Baggins in the quest of the One Ring. She became the mother of Frodo Gardner, Elanor, Rose, Merry, Pippin, Goldilocks, Hamfast, Daisy, Primrose, Bilbo, Ruby, Robin, and Tolman (Tom).

Biography
Rosie was a Cotton; the daughter of Tolman Cotton and Lily Brown, and the sister of Tolman (Tom), Wilcome Cotton (Jolly)


In [9]:
print(text[200:500])

 quest of the One Ring. She became the mother of Frodo Gardner, Elanor, Rose, Merry, Pippin, Goldilocks, Hamfast, Daisy, Primrose, Bilbo, Ruby, Robin, and Tolman (Tom).

Biography
Rosie was a Cotton; the daughter of Tolman Cotton and Lily Brown, and the sister of Tolman (Tom), Wilcome Cotton (Jolly)


In [10]:
def create_embeddings(text, model="text-embedding-004", task_type="SEMANTIC_SIMILARITY"):
    return client.models.embed_content(model=model, contents=text, config=types.EmbedContentConfig(task_type=task_type))

In [11]:
embeddings = create_embeddings(chunks)
len(embeddings.embeddings)

55

In [12]:
len(embeddings.embeddings)

embeddings.embeddings[0].values[:10]

[-0.054459885,
 0.050495975,
 -0.01605832,
 0.055216204,
 0.054865774,
 0.082677625,
 -0.046905264,
 -0.025302313,
 0.02983876,
 -0.06661411]

In [13]:
def cosine_similarity(vec1, vec2):  
    return np.dot(vec1, vec2) / (np.linalg.norm(vec1) * np.linalg.norm(vec2))

In [14]:
def semantic_search(query, chunks, embeddings, k=5):
    query_embedding = create_embeddings(query).embeddings[0].values 
    similarity_scores = []
    
    for i, chunk_embedding in enumerate(embeddings.embeddings):
        similarity_score = cosine_similarity(query_embedding, chunk_embedding.values)
        similarity_scores.append((i, similarity_score))

    similarity_scores.sort(key=lambda x: x[1], reverse=True)
    top_indices = [index for index, _ in similarity_scores[:k]]
    
    return [chunks[index] for index in top_indices]

In [15]:
fråga = "Vem va Samwise Gamgee?"
svar = semantic_search(fråga, chunks, embeddings, k=1)
print(svar) 

['. He lived with his father, Hamfast Gamgee, known commonly as "The Gaffer", on Bagshot Row in the Shire, close to Bag End. He had five siblings: Hamson, Halfred, Daisy, May, and Marigold.\n\nQuest of the Ring\nAs "punishment" for eavesdropping on Gandalf\'s conversation with Frodo regarding the dangers of the One Ring while trimming the grass of Bag End, Gandalf chose Sam to be Frodo\'s companion on his journey to Rivendell. Sam saved Frodo\'s life more than once during the quest to destroy the Ring, ']


In [16]:
system_prompt = """Jag kommer ställa dig en fråga, och jag vill att du svarar
baserat bara på kontexten jag skickar med, och ingen annan information.
Om det inte finns nog med information i kontexten för att svara på frågan,
säg "Det vet jag inte". Försök inte att gissa.
Formulera dig enkelt och dela upp svaret i fina stycken. """

In [17]:
def generate_user_prompt(query):
    context = "\n".join(semantic_search(query, chunks, embeddings))
    user_prompt = f"Frågan är {query}. Här är kontexten: {context}."
    return user_prompt

In [18]:
def generate_response(system_prompt, user_message, model="gemini-2.0-flash"):
    response = client.models.generate_content(
        model=model,
        config=genai.types.GenerateContentConfig(
            system_instruction=system_prompt),
            contents=generate_user_prompt(user_message)
        )
    return response

In [19]:
print(generate_response(system_prompt, "Vem är Samwise Gamgee?").text)

Samwise Gamgee, även känd som Samwise Gardner eller helt enkelt som Sam, var en hobbit från Fylke.

Han var Frodo Baggins trädgårdsmästare och bästa vän.

Han bodde med sin far, Hamfast Gamgee, känd som "The Gaffer", på Bagshot Row i Fylke, nära Bag End. Han hade fem syskon: Hamson, Halfred, Daisy, May och Marigold.

Han var den mest lojala medlemmen i Ringens brödraskap och spelade en avgörande roll i att skydda Frodo.



In [20]:
fråga = "Vem var Samwise Gamgees fru?"
svar = generate_response(system_prompt, fråga).text
print(svar)

Rosie var Samwise Gamgees fru.



In [21]:
validation_data = [
  {
    "question": "Vem var Rosie Cottons man?",
    "ideal_answer": """Samvise Gamgee"""
  }
]

print(validation_data[0]["question"])
print()
print(validation_data[0]["ideal_answer"])

Vem var Rosie Cottons man?

Samvise Gamgee


In [22]:
evaluation_system_prompt = """Du är ett intelligent utvärderingssystem vars uppgift är att utvärdera en AI-assistents svar. 
Om svaret är väldigt nära det önskade svaret, sätt poängen 1. Om svaret är felaktigt eller inte bra nog, sätt poängen 0.
Om svaret är delvis i linje med det önskade svaret, sätt poängen 0.5. Motivera kort varför du sätter den poäng du gör.
"""

In [23]:
query = validation_data[0]["question"]

response = generate_response(system_prompt, query)

evaluation_prompt = f"""Frågan är: {query}
AI-assistantens svar är: {response.text}
Det önskade svaret är: {validation_data[0]["ideal_answer"]}"""

evaluation_response = generate_response(evaluation_system_prompt, evaluation_prompt)
print(evaluation_response.text)


Poäng: 1.
Motivering: Svaret är korrekt och välformulerat.
